In [ ]:
!pip install langchain
!pip install langchain-groq

import os
import json
import requests
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_groq import ChatGroq
from langchain.agents import create_tool_calling_agent, AgentExecutor

In [ ]:
from google.colab import userdata

SECTORS_API_KEY = userdata.get('SECTORS_API_KEY')
GROQ_API_KEY = userdata.get('GROQ_API_KEY')

# Function

In [ ]:
from langchain_core.tools import tool

def retrieve_from_endpoint(url: str) -> dict:
    headers = {"Authorization": SECTORS_API_KEY}

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        data = response.json()
    except requests.exceptions.HTTPError as err:
        raise SystemExit(err)
    return json.dumps(data)

def get_today_date() -> str:
    """
    Get today's date
    """
    from datetime import date

    today = date.today()
    return today.strftime("%Y-%m-%d")

## Tool

In [ ]:
@tool
def get_top_companies_by_tx_volume(
    start_date: str, end_date: str, top_n: int = 5
) -> str:
    """
    Get top companies by transaction volume
    """
    url = f"https://api.sectors.app/v1/most-traded/?start={start_date}&end={end_date}&n_stock={top_n}"

    return retrieve_from_endpoint(url)

@tool
def get_company_overview(stock: str) -> str:
    """
    Get the overview of a stock, including email, address, phone number, market cap information, listing date etc
    """
    url = f"https://api.sectors.app/v1/company/report/{stock}/?sections=overview"
    print(url)

    return retrieve_from_endpoint(url)


@tool
def get_daily_tx(stock: str, start_date: str, end_date: str) -> str:
    """
    Get daily transaction for a stock
    """
    url = f"https://api.sectors.app/v1/daily/{stock}/?start={start_date}&end={end_date}"
    print(url)

    return retrieve_from_endpoint(url)


@tool
def get_performance_since_ipo(stock: str) -> str:
  """
  Get stock performance after IPO listing
  """
  url = f"https://api.sectors.app/v1/listing-performance/{stock}/"
  print(url)

  return retrieve_from_endpoint(url)

# Pipeline

In [ ]:
tools = [
    get_company_overview,
    get_top_companies_by_tx_volume,
    get_daily_tx,
    get_performance_since_ipo
]

llm = ChatGroq(
    temperature=0,
    model_name="llama3-groq-70b-8192-tool-use-preview",
    groq_api_key=GROQ_API_KEY,
)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            f"""
            You are the financial robo-advisor, answer the following queries like a Financial Agent,
            being as factual and analytical as possible. Do not make up stuff.
            Try to cite your number as much as possible. Retrieve and show the corresponding data first
            such as the market cap number or data that correspond to the question before you make conclusion and show the answer.
            Your get_daily_tx tool will need the start and end dates, and if you need the start and end dates
            and they are not explicitly provided, please infer from the query.
            If the volume was about a single day (such as specific single date or yesterday),
            the start date and end date parameter should be the same date. Start and end date will be in the format YYYY-MM-DD.
            Whenever you return a list of names, return also the corresponding values for each name.

            For most of your queries such as company performance since the date of its initial public offering (IPO),
            these information should be available in your get_performance_since_ipo tool,
            but the listing performance data is accessible only for stocks listed after May 2005.
            Note that the endpoint for performance since IPO has only one required parameter, which is the stock.
            Every time you return the symbol of the stock, result also the number in question in a nice tabular format.

            Today's date is {get_today_date()}.
            """
        ),
        ("human", "{input}"),
        # msg containing previous agent tool invocations and corresponding tool outputs
        MessagesPlaceholder("agent_scratchpad"),
    ]
)

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# Streamlit